In [ ]:
import torch 
import numpy as np
from torch import nn 
import gymnasium as gym
from torch.distributions.categorical import Categorical
from torch.optim import Adam

In [ ]:
## class MLP = neural policy 
# Function: 
## get policy : output action distribution 
## get action : sample from the policy 
## compute loss: compute the loss function 
## train : loop of epoch : act, collect obs, backprobagate, optimize 

class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        module = [
            nn.Linear(input_size,64),
            nn.ReLU(inplace=True),
            nn.Linear(64,output_size)
        ]
        self.mlp= nn.Sequential(*module)

    def forward(self, state):
        return self.mlp(state)
    
def train(env_name='CartPole-v0', lr=1e-2, batch_size=5000, epochs=50, render=False): 
    env = gym.make(env_name)
    obs_size = env.observation_space.shape[0]
    act_size = env.action_space.n
    logit_net = MLP(obs_size,act_size)

    def get_policy(obs): 
        return Categorical(logits=logit_net(obs))
    
    def get_action(obs):
        return get_policy(obs).sample().item()
    
    def compute_loss(acts, obss, weights):
        prob = get_policy(obss).log_prob(acts)
        return - (weights * prob).mean()
    

    optimizer = Adam(params=logit_net.parameters(), lr=lr)

    def epoch_train():
        batch_obs= []
        batch_act = []
        batch_weight = []
        batch_return = []
        batch_len = []
        ep_reward = []

        obs = env.reset()
        done = False 
        

        while True : 
            # collect
            batch_obs.append(obs.copy())
            act = get_action(obs)
            batch_act.append(act)
            obs, reward, done = env.step(act)

            ep_reward.append(reward)


            if done: 
                # episode over collect info about episode 
                ep_return, ep_len = sum(ep_reward), len(ep_reward)
                batch_return.append(ep_return)
                batch_len. append
                batch_weight += [ep_return] * ep_len

                obs, done, ep_reward  =  env.reset(), False, [] 

                if len(batch_obs)> batch_size:
                    break
        

        loss = compute_loss(acts=torch.as_tensor(batch_act), obss=torch.as_tensor(batch_obs), weights=torch.as_tensor(batch_weight))
        loss.backward()
        optimizer.step()
        
        return loss, batch_return, batch_len
    

    for i in range(epochs):
        _,_,_ = epoch_train()

if __name__ == '__main__':
    train()

/home/yanser/Documents/ai-notebook/venv/lib/python3.12/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


NameError: name 'Adam' is not defined